In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
import data_preprocessing
import importlib
importlib.reload(data_preprocessing)
import cross_validation
import TEST
importlib.reload(TEST)
importlib.reload(cross_validation)
import implemented_functions
importlib.reload(implemented_functions)



<module 'implemented_functions' from '/Users/mpecaut/ML_project1/project1/implemented_functions.py'>

In [2]:
np.random.seed(42)

In [5]:
#Load Data
x_train_raw, x_test_raw,y_train_raw, train_ids, test_ids = load_csv_data('dataset')

In [6]:
print(x_train_raw.shape)
print(x_test_raw.shape)


(328135, 321)
(109379, 321)


In [18]:
x_train_raw

array([[5.3000000e+01, 1.1000000e+01, 1.1162015e+07, ...,           nan,
                  nan, 2.0000000e+00],
       [3.3000000e+01, 1.2000000e+01, 1.2152015e+07, ...,           nan,
                  nan,           nan],
       [2.0000000e+01, 1.0000000e+01, 1.0202015e+07, ..., 1.0000000e+00,
        2.0000000e+00, 2.0000000e+00],
       ...,
       [3.9000000e+01, 1.0000000e+01, 1.0202015e+07, ..., 2.0000000e+00,
        2.0000000e+00, 2.0000000e+00],
       [3.3000000e+01, 1.2000000e+01, 1.2302015e+07, ...,           nan,
                  nan, 2.0000000e+00],
       [3.2000000e+01, 9.0000000e+00, 9.1220150e+06, ...,           nan,
                  nan, 2.0000000e+00]], shape=(328135, 321))

In [7]:
#Reformat y_train to have 0's instead of -1's
y_train_original = y_train_raw.copy()
y_train_raw[y_train_raw == -1] = 0

In [ ]:
#replace NaN by a float to treat NaN as a categorical feature
#TODO: NAN'S :apagnan -10 or 0 --> SAME BETWEEN TRAIN AND TEST 
x_test_raw = np.nan_to_num(x_test_raw, nan = -10.0)
x_test_raw.shape
x_test_raw

array([[ 4.4000000e+01,  2.0000000e+00,  2.0820150e+06, ...,
         1.0000000e+00,  1.0000000e+00,  2.0000000e+00],
       [ 2.7000000e+01,  1.0000000e+00,  1.1920150e+06, ...,
        -1.0000000e+01, -1.0000000e+01,  2.0000000e+00],
       [ 3.5000000e+01,  5.0000000e+00,  5.2620150e+06, ...,
         1.0000000e+00,  1.0000000e+00,  2.0000000e+00],
       ...,
       [ 9.0000000e+00,  1.1000000e+01,  1.1272015e+07, ...,
         9.0000000e+00,  9.0000000e+00, -1.0000000e+01],
       [ 1.5000000e+01,  1.2000000e+01,  1.2122015e+07, ...,
         1.0000000e+00,  1.0000000e+00,  2.0000000e+00],
       [ 4.5000000e+01,  1.2000000e+01,  1.2282015e+07, ...,
        -1.0000000e+01, -1.0000000e+01,  2.0000000e+00]],
      shape=(109379, 321))

### One Hot Encode

In [9]:
data_annoted = data_preprocessing.read_annotated_csv('dataset/data_anotated.csv')

In [10]:
x_train, y_train, x_test = data_preprocessing.preprocess_data2(x_train_raw[:50000,:], y_train_raw[:50000], x_test_raw[:50000,:], data_annoted)


/Users/mpecaut/ML_project1/project1/data_preprocessing.py:172: RuntimeWarning: invalid value encountered in divide
  data = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))


In [ ]:
x_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
"""x_train_filtered = data_preprocessing.remove_useless(x_train_raw, data_annoted)
x_test_filtered = data_preprocessing.remove_useless(x_test_raw, data_annoted)"""

'x_train_filtered = data_preprocessing.remove_useless(x_train_raw, data_annoted)\nx_test_filtered = data_preprocessing.remove_useless(x_test_raw, data_annoted)'

In [ ]:
"""data_train = data_preprocessing.clean_data(x_train_filtered[:100000,:], data_annoted)
data_test = data_preprocessing.clean_data(x_test_filtered[:100000,:], data_annoted)"""

'data_train = data_preprocessing.clean_data(x_train_filtered[:100000,:], data_annoted)\ndata_test = data_preprocessing.clean_data(x_test_filtered[:100000,:], data_annoted)'

In [22]:
np.savetxt('x_train_preprocessed.csv', x_train, delimiter=',')
np.savetxt('y_train_preprocessed.csv', y_train, delimiter=',')
np.savetxt('x_test_preprocessed.csv', x_test, delimiter=',')

In [7]:
x_train = np.loadtxt('x_train_preprocessed.csv', delimiter=',')
x_test = np.loadtxt('x_test_preprocessed.csv', delimiter=',')
y_train = np.loadtxt('y_train_preprocessed.csv', delimiter=',')

In [83]:
x_train.shape

(50000, 576)

In [8]:

initial_w = np.zeros(x_train.shape[1])
max_iters = 50

x_test = x_train[45000:,:]
y_test = y_train[45000:]
x_train = x_train[:45000,:]
y_train = y_train[:45000]


w_least_squares, loss_ls = implemented_functions.least_squares(y_train, x_train)
print(f"Least Squares: Loss = {loss_ls}")

best_mse_coef = 0
best_mse_loss = np.inf
best_mse_weights = None
for gamma in [0.01, 0.1, 0.5, 1]:
    w_sgd, loss_sgd = implemented_functions.mean_squared_error_sgd(y_train,  x_train, initial_w, max_iters, gamma)
    if loss_sgd[-1] < best_mse_loss:
        best_mse_loss = loss_sgd[-1]
        best_mse_coef = gamma
        best_mse_weights = w_sgd
    print(f"Mean Squared Error SGD (gamma={gamma}): Loss = {loss_sgd}")

best_log_lasso_coef = 0
best_log_lasso_loss = np.inf
best_log_lasso_weights = None
for gamma in [0.01, 0.1, 0.5]:
    for lambda_ in [0.1, 0.01]:
        w_log_lasso, loss_log_lasso = implemented_functions.reg_logistic_lasso_subgradient(y_train, x_train, lambda_, initial_w, max_iters, gamma)
        if loss_log_lasso < best_log_lasso_loss:
            best_log_lasso_loss = loss_log_lasso
            best_log_lasso_weights = w_log_lasso
        print(f"Regularized Lasso Logistic Regression (gamma={gamma}, lambda={lambda_}): Loss = {loss_log_lasso}")
        
best_ridge_coef = 0
best_ridge_loss = np.inf
best_ridge_weights = None
for ridge_coef in [0.001, 0.01, 0.1, 1, 10]:
    w_ridge, loss_ridge = implemented_functions.ridge_regression(y_train, x_train, ridge_coef)
    if loss_ridge < best_ridge_loss:
        best_ridge_loss = loss_ridge
        best_ridge_coef = ridge_coef
        best_ridge_weights = w_ridge
    print(f"Ridge Regression (alpha={ridge_coef}): Loss = {loss_ridge}")

best_log_gamma = 0
best_log_loss = np.inf
best_log_weights = None
for gamma in [0.01, 0.1, 0.5, 1]:
    w_logistic, loss_logistic = implemented_functions.logistic_regression(y_train, x_train, initial_w, max_iters, gamma)
    if loss_logistic < best_log_loss:
        best_log_loss = loss_logistic
        best_log_gamma = gamma
        best_log_weights = w_logistic
    print(f"Logistic Regression: Loss = {loss_logistic}")

best_log_reg_coef = 0
best_log_reg_loss = np.inf
best_log_reg_weights = None
for gamma in [0.01, 0.1]:
    for reg_coef in [0.001, 0.01, 0.1]:
        w_reg_logistic, loss_reg_logistic = implemented_functions.reg_logistic_regression(y_train, x_train, reg_coef, initial_w, max_iters, gamma)
        if loss_reg_logistic < best_log_reg_loss:
            best_log_reg_loss = loss_reg_logistic
            best_log_reg_coef = reg_coef
            best_log_reg_weights = w_reg_logistic
        print(f"Regularized Logistic Regression (alpha={reg_coef}, gamma={gamma}): Loss = {loss_reg_logistic}")



/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:140: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(tx,y)[0]


Least Squares: Loss = 0.03209577896705887
Mean Squared Error SGD (gamma=0.01): Loss = [0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.04345555555555556, 0.10460163310080063, 0.07674929675650492, 0.05510969595540356, 0.04592840657268578, 0.04592461697787984, 0.047647450453146244, 0.04840684037558661, 0.050707335918181454, 0.05577344593513922, 0.04997650457769179, 0.05375440864871297, 0.046217230162049414, 0.049233421680005364, 0.04753471636636235, 0.04705758883254935, 0.04725311059930179, 0.047679788835154585, 0.048366736445749174, 0.0467584773446901, 0.04642375098896919, 0.04549349827716162, 0.04755884514355169, 0.04681175708961924, 0.047234988527297427, 0.047538788512213, 0.04732329008213579, 0.04617092360163797, 0.046334533946144674, 0.047980887253537795, 0.04483021906156473, 0.04529905142728463, 0.04735922290637097, 0.04

In [16]:

import implemented_functions

def test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights, threshold=0.5):
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, w_sgd, "SGD", threshold)
    print(f"F1 score SGD is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, w_least_squares, "Least Squares", threshold)
    print(f"F1 score Least Squares is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, best_ridge_weights, "Ridge", threshold)
    print(f"F1 score Ridge is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, best_log_weights, "Logistic", threshold)
    print(f"F1 score Logistic is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, best_log_reg_weights, "Regularized Logistic", threshold)
    print(f"F1 score Regularized Logistic is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, best_log_lasso_weights, "Regularized Lasso", threshold)
    print(f"F1 score Regularized Lasso is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    return y_pred

def test_thresholds(x_test, y_test, weights, method):
    for threshold in [0.3, 0.5, 0.7, 0.8, 0.9]:
        accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, weights, method, threshold)
        print(f"-----{method}: F1 score {method} at threshold {threshold} is {implemented_functions.compute_f1_score(y_test, y_pred)}")

test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights, threshold=0.5)
test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights, threshold=0.3)
test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights, threshold=0.7)

test_thresholds(x_test, y_test, best_ridge_weights, "Ridge")



Accuracy of SGD is 91.64%
F1 score SGD is 0.0
Accuracy of Least Squares is 91.97999999999999%
F1 score Least Squares is 0.12253829321663019
Accuracy of Ridge is 76.58%
F1 score Ridge is 0.36185286103542236
[0.18878668 0.01735786 0.05975914 ... 0.00851406 0.44185317 0.07404378]
Accuracy of Logistic is 90.66%
F1 score Logistic is 0.33946251768033947
[0.09847767 0.09479492 0.08190276 ... 0.08479488 0.15641836 0.11541087]
Accuracy of Regularized Logistic is 91.64%
F1 score Regularized Logistic is 0.0
[0.11006322 0.10422189 0.09222174 ... 0.09536845 0.17423382 0.12823048]
Accuracy of Regularized Lasso is 91.64%
F1 score Regularized Lasso is 0.0
Accuracy of SGD is 91.64%
F1 score SGD is 0.0
Accuracy of Least Squares is 87.52%
F1 score Least Squares is 0.007142857142857143
Accuracy of Ridge is 52.76%
F1 score Ridge is 0.49810366624525915
[0.18878668 0.01735786 0.05975914 ... 0.00851406 0.44185317 0.07404378]
Accuracy of Logistic is 83.04%
F1 score Logistic is 0.32941176470588235
[0.09847767 0

In [8]:
w1 = np.sum(y_test == 1)
w1

832

In [ ]:
x_train = np.loadtxt('x_train_preprocessed.csv', delimiter=',')
x_test = np.loadtxt('x_test_preprocessed.csv', delimiter=',')
y_train = np.loadtxt('y_train_preprocessed.csv', delimiter=',')
y_pred = x_test@best_ridge_weights


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 575 is different from 561)

In [ ]:
best_ridge_weights.shape

(575,)

In [ ]:
x_train.shape

(8658, 575)

In [ ]:
x_test.shape

(50000, 561)